## Week 5 Part 2 Assignment:
## CUNY MSDS DATA620 - Web Analytics
---
### Team5: Christopher Estevez, Meaghan Burke, Rickidon Singh,  Ritesh Lohiya, Rose Koh
### 07/16/2018 (due date)
##### python version: ?
---

## Document Classification


It can be useful to be able to classify new "test" documents using already classified "training" documents.  A common example is using a corpus of labeled spam and ham (non-spam) e-mails to predict whether or not a new document is spam.  Here is one example of such data:  http://archive.ics.uci.edu/ml/datasets/Spambase

For this project, you can either use the above dataset to predict the class of new documents (either withheld from the training dataset or from another source such as your own spam folder).

For more adventurous students, you are welcome (encouraged!) to come up a different set of documents (including scraped web pages!?) that have already been classified (e.g. tagged), then analyze these documents to predict how new documents should be classified.

This assignment is due end of day on Monday, July 16th.  You may work in a small team if you want.

In [178]:
import nltk
import urllib.request
import pandas as pd
import numpy as np
from sklearn import ensemble
from sklearn import svm
import sklearn.metrics as sm
import matplotlib.pyplot as plt
import re
import string
%matplotlib inline


In [179]:
names = ["word_freq_order", "word_freq_mail", "word_freq_receive", "word_freq_will",
        "word_freq_people", "word_freq_report", "word_freq_addresses", 
        "word_freq_free", "word_freq_business", "word_freq_email", "word_freq_you", 
        "word_freq_credit", "word_freq_your", "word_freq_font", "word_freq_000", 
        "word_freq_money", "word_freq_hp", "word_freq_hpl", "word_freq_george",
        "word_freq_650", "word_freq_lab", "word_freq_labs", "word_freq_telnet", 
        "word_freq_857", "word_freq_data", "word_freq_415", "word_freq_85", 
        "word_freq_technology", "word_freq_1999", "word_freq_parts", "word_freq_pm",
        "word_freq_direct", "word_freq_cs", "word_freq_meeting", "word_freq_original",
        "word_freq_project", "word_freq_re", "word_freq_edu", "word_freq_table",
        "word_freq_conference", "char_freq_;", "char_freq_(", "char_freq_[", 
        "char_freq_!", "char_freq_$", "char_freq_#", "capital_run_length_average", 
        "capital_run_length_longest", "capital_run_length_total", "spam"
        ]

In [180]:
spam_data = pd.read_csv("https://raw.githubusercontent.com/silverrainb/web-analytics/master/week5/spambase.csv", names = names)

In [181]:
spam_data.describe()


word_freq_order  word_freq_mail  word_freq_receive  word_freq_will  \
count      4601.000000     4601.000000        4601.000000     4601.000000   
mean          0.090067        0.239413           0.059824        0.541702   
std           0.278616        0.644755           0.201545        0.861698   
min           0.000000        0.000000           0.000000        0.000000   
25%           0.000000        0.000000           0.000000        0.000000   
50%           0.000000        0.000000           0.000000        0.100000   
75%           0.000000        0.160000           0.000000        0.800000   
max           5.260000       18.180000           2.610000        9.670000   

       word_freq_people  word_freq_report  word_freq_addresses  \
count       4601.000000       4601.000000          4601.000000   
mean           0.093930          0.058626             0.049205   
std            0.301036          0.335184             0.258843   
min            0.000000          0.000000             0.000000   
25%            0.000000          0.000000             0.000000   
50%            0.000000          0.000000             0.000000   
75%            0.000000          0.000000             0.000000   
max            5.550000         10.000000             4.410000   

       word_freq_free  word_freq_business  word_freq_email     ...       \
count     4601.000000         4601.000000      4601.000000     ...        
mean         0.248848            0.142586         0.184745     ...        
std          0.825792            0.444055         0.531122     ...        
min          0.000000            0.000000         0.000000     ...        
25%          0.000000            0.000000         0.000000     ...        
50%          0.000000            0.000000         0.000000     ...        
75%          0.100000            0.000000         0.000000     ...        
max         20.000000            7.140000         9.090000     ...        

       char_freq_;  char_freq_(  char_freq_[  char_freq_!  char_freq_$  \
count  4601.000000  4601.000000  4601.000000  4601.000000  4601.000000   
mean      0.038575     0.139030     0.016976     0.269071     0.075811   
std       0.243471     0.270355     0.109394     0.815672     0.245882   
min       0.000000     0.000000     0.000000     0.000000     0.000000   
25%       0.000000     0.000000     0.000000     0.000000     0.000000   
50%       0.000000     0.065000     0.000000     0.000000     0.000000   
75%       0.000000     0.188000     0.000000     0.315000     0.052000   
max       4.385000     9.752000     4.081000    32.478000     6.003000   

       char_freq_#  capital_run_length_average  capital_run_length_longest  \
count  4601.000000                 4601.000000                 4601.000000   
mean      0.044238                    5.191515                   52.172789   
std       0.429342                   31.729449                  194.891310   
min       0.000000                    1.000000                    1.000000   
25%       0.000000                    1.588000                    6.000000   
50%       0.000000                    2.276000                   15.000000   
75%       0.000000                    3.706000                   43.000000   
max      19.829000                 1102.500000                 9989.000000   

       capital_run_length_total         spam  
count               4601.000000  4601.000000  
mean                 283.289285     0.394045  
std                  606.347851     0.488698  
min                    1.000000     0.000000  
25%                   35.000000     0.000000  
50%                   95.000000     0.000000  
75%                  266.000000     1.000000  
max                15841.000000     1.000000  

[8 rows x 50 columns]

In [182]:
count_spam = len(spam_data[spam_data.spam==1])
count_nonspam = len(spam_data[spam_data.spam==0])
print(count_spam) 
print(count_nonspam) 

1813
2788


In [183]:
#Sampling the data into train test and validation sets
sampling = np.random.choice(("train", "test", "val"), p=[0.7, 0.15, 0.15], size=len(spam_data))


In [184]:
train_set, test_set, val_set =  spam_data[sampling == "train"], spam_data[sampling == "test"],spam_data[sampling == "val"]
print(len(train_set))
print(len(test_set))
print(len(val_set))
print(len(train_set)+len(test_set)+len(val_set))


3171
712
718
4601


In [185]:
# Build confusion matrix function
def cm_metrics(Actual, Predicted):
    cm = sm.confusion_matrix(Actual, Predicted, labels=[1, 0])
    print("True positives: %d" %cm[0,0])
    print("False positives: %d" %cm[1,0])
    print("True negatives: %d" %cm[1,1])
    print("False negatives: %d" %cm[0,1])
    print
    print(sm.classification_report(Actual, Predicted, labels=[1,0], target_names=["Spam", "Ham"]))

First we will use Randon Forest Method

In [186]:
# Train the classifier usinf Random Forest method
train_tar = train_set['spam']
train_dat = train_set.drop(labels='spam', axis=1)

randon_forest = ensemble.RandomForestClassifier(criterion="entropy", random_state=1)
randon_forest_fit = randon_forest.fit(train_dat, train_tar)

randon_forest_train = randon_forest_fit.predict(train_dat)
cm_metrics(train_tar, randon_forest_train)

True positives: 1228
False positives: 1
True negatives: 1927
False negatives: 15
             precision    recall  f1-score   support

       Spam       1.00      0.99      0.99      1243
        Ham       0.99      1.00      1.00      1928

avg / total       0.99      0.99      0.99      3171



In [187]:
#Validation data
val_tar = val_set['spam']
val_dat = val_set.drop(labels='spam', axis=1)

randon_forest_val = randon_forest_fit.predict(val_dat)
cm_metrics(val_tar, randon_forest_val)

True positives: 261
False positives: 19
True negatives: 416
False negatives: 22
             precision    recall  f1-score   support

       Spam       0.93      0.92      0.93       283
        Ham       0.95      0.96      0.95       435

avg / total       0.94      0.94      0.94       718



In [188]:
#test data
test_tar = test_set['spam']
test_dat = test_set.drop(labels='spam', axis=1)

randon_forest_test = randon_forest_fit.predict(test_dat)
cm_metrics(test_tar, randon_forest_test)

True positives: 259
False positives: 16
True negatives: 409
False negatives: 28
             precision    recall  f1-score   support

       Spam       0.94      0.90      0.92       287
        Ham       0.94      0.96      0.95       425

avg / total       0.94      0.94      0.94       712



In [203]:
# Accuracy = (TP+FN)/(TP+TN+FP+FN)
Accuarcy = (259+409)/(259+16+409+28)
Accuarcy

0.9382022471910112

SVM Method:

In [189]:
#Lets try SVM
train_tar = train_set['spam']
train_dat = train_set.drop(labels='spam', axis=1)
svm = svm.SVC(random_state=1)
svm_fit = svm.fit(train_dat, train_tar)

svm_train = svm_fit.predict(train_dat)
cm_metrics(train_tar, svm_train)

True positives: 1121
False positives: 53
True negatives: 1875
False negatives: 122
             precision    recall  f1-score   support

       Spam       0.95      0.90      0.93      1243
        Ham       0.94      0.97      0.96      1928

avg / total       0.95      0.94      0.94      3171



In [190]:
#Validation data
val_tar = val_set['spam']
val_dat = val_set.drop(labels='spam', axis=1)

svm_val = svm_fit.predict(val_dat)
cm_metrics(val_tar, svm_val)

True positives: 232
False positives: 82
True negatives: 353
False negatives: 51
             precision    recall  f1-score   support

       Spam       0.74      0.82      0.78       283
        Ham       0.87      0.81      0.84       435

avg / total       0.82      0.81      0.82       718



In [191]:
#test data
test_tar = test_set['spam']
test_dat = test_set.drop(labels='spam', axis=1)

svm_test = svm_fit.predict(test_dat)
cm_metrics(test_tar, svm_test)

True positives: 219
False positives: 66
True negatives: 359
False negatives: 68
             precision    recall  f1-score   support

       Spam       0.77      0.76      0.77       287
        Ham       0.84      0.84      0.84       425

avg / total       0.81      0.81      0.81       712



In [202]:
# Accuracy = (TP+FN)/(TP+TN+FP+FN)
Accuarcy = (219+359)/(219+66+359+68)
Accuarcy

0.8117977528089888

We will try now try to classify using Decision Tree

Decision Tree

In [205]:
#Lets us see the accuracy using Decision Tree
train_tar = train_set['spam']
train_dat = train_set.drop(labels='spam', axis=1)

from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(class_weight = {0:2,1:1})
tree_fit = tree.fit(train_dat, train_tar)

tree_train = tree_fit.predict(train_dat)
cm_metrics(train_tar, tree_train)

True positives: 1241
False positives: 0
True negatives: 1928
False negatives: 2
             precision    recall  f1-score   support

       Spam       1.00      1.00      1.00      1243
        Ham       1.00      1.00      1.00      1928

avg / total       1.00      1.00      1.00      3171



In [206]:
#Validation data
val_tar = val_set['spam']
val_dat = val_set.drop(labels='spam', axis=1)

tree_val = tree_fit.predict(val_dat)
cm_metrics(val_tar, tree_val)

True positives: 253
False positives: 30
True negatives: 405
False negatives: 30
             precision    recall  f1-score   support

       Spam       0.89      0.89      0.89       283
        Ham       0.93      0.93      0.93       435

avg / total       0.92      0.92      0.92       718



In [207]:
#test data
test_tar = test_set['spam']
test_dat = test_set.drop(labels='spam', axis=1)

tree_test = tree_fit.predict(test_dat)
cm_metrics(test_tar, tree_test)

True positives: 257
False positives: 38
True negatives: 387
False negatives: 30
             precision    recall  f1-score   support

       Spam       0.87      0.90      0.88       287
        Ham       0.93      0.91      0.92       425

avg / total       0.91      0.90      0.90       712



In [208]:
# Accuracy = (TP+FN)/(TP+TN+FP+FN)
Accuarcy = (257+387)/(257+38+387+30)
Accuarcy

0.9044943820224719

Conclusions:

Out of the three methods we tried Random Forest has the best accuracy rate(94%) compared to SVM(